In [1]:
pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:00


In [2]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import json
from base64 import b64encode, b64decode
from pprint import pprint
import pickle

In [6]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import json
from base64 import b64encode, b64decode
from pprint import pprint
import pickle



class ITKoin:
    def __init__ (self):
        self.pending_transactions_filename = 'pending_01.txt'
        self.chain_filename = 'chain_01.txt'
        self.ITKoin_users = ['Antal', 'Béla', 'Cili']  
        self.ICO = 100
        self.chain = []
        self.pending_transactions = []
        self.unspent_outputs = []
        self.payer_unspent_outputs = []

    def create_signature (self, data, signer_privatekey):
        signatureobject = pkcs1_15.new(signer_privatekey) # signature objektum létrehozása
        hashobject = self.create_hashobject(data) # hash objektum létrehozása
        signaturevalue = signatureobject.sign(hashobject) # aláírás értékének elkészítése
        # pprint(signaturevalue)
        b64signaturevalue = b64encode(signaturevalue) # base64 kódolás
        # pprint(b64signaturevalue)
        # pprint(b64signaturevalue.decode())
        return b64signaturevalue.decode()

    def verify_signature(self, data, b64signaturevalue, rsapublickey):
        verifyobject = pkcs1_15.new(rsapublickey) # verify objektum létrehozása
        hashobject = self.create_hashobject(data) # hash objektum létrehozása
        signaturevalue = b64decode(b64signaturevalue.encode()) # aláírás érték base64 dekódolása
        signatureerror = verifyobject.verify(hashobject, signaturevalue) # aláírás ellenőrzése
        validsignature = not signatureerror # értéke: True, ha az aláírás érvényes
        return validsignature

    @staticmethod
    def create_hashobject (data):
        stringdump = json.dumps(data)
        hashobject = SHA256.new(stringdump.encode())
        return hashobject

    @staticmethod
    def create_hashhexvalue (data):
        stringdump = json.dumps(data)
        hashobject = SHA256.new(stringdump.encode())
        return hashobject.hexdigest()

    @staticmethod
    def save_list(list, filename):
        f = open(filename, 'wb')
        pickle.dump(list, f)
        f.close()
        return

    def create_users(self):
        for user in self.ITKoin_users:
            self.generate_rsa_key(user)
        return

    def generate_rsa_key(self, username): # a username lesz a filenév töve és három file-t generál: a privát és publikus kulcsoknak, ill. az ID-nak
        key = RSA.generate(2048)
        publickey = key.publickey()
        privatekey_filename = username + '_priv.pem'
        f = open(privatekey_filename, 'wb')
        f.write(key.export_key())
        f.close()
        publickey_filename = username + '_pub.pem'
        f = open(publickey_filename, 'wb')
        f.write(publickey.export_key())
        f.close()
        publickey_string = publickey.export_key().decode('ascii') # bináris stringet karakter stringgé konvertáljuk, hogy a json.dumps működjön rajta
        recipient_id_filename = username + '_id.txt'
        f = open(recipient_id_filename, 'wb')
        f.write(self.create_hashhexvalue(publickey_string).encode('ascii')) # a hexa string hash értéket bináris stringgé konvertáljuk a file-ba íráshoz
        f.close()
        return

    @staticmethod
    def load_privatekey (username):
        privatekey_filename = username + '_priv.pem'
        fileobject = open(privatekey_filename, 'r')
        user_privatekey = RSA.import_key(fileobject.read())
        #pprint(user_privatekey)
        return user_privatekey

    @staticmethod
    def load_publickey (username):
        publickey_filename = username + '_pub.pem'
        fileobject = open(publickey_filename, 'r')
        user_publickey = RSA.import_key(fileobject.read())
        #pprint(user_publickey)
        return user_publickey

    @staticmethod
    def load_publickeyfromprivatekey (username):
        privatekey_filename = username + '_priv.pem'
        fileobject = open(privatekey_filename, 'r')
        user_privatekey = RSA.import_key(fileobject.read())
        user_publickey = user_privatekey.publickey()
        #pprint(user_publickey)
        return user_publickey

    @staticmethod
    def load_id (username):
        id_filename = username + '_id.txt'
        fileobject = open(id_filename, 'r')
        user_id = fileobject.read()
        #pprint(user_id)
        return user_id

    def load_chain(self):
      #bányász csinálja
        fileobject = open(self.chain_filename, 'rb')
        self.chain = pickle.load(fileobject)
        # pprint(self.chain)
        # Itt kell ellenőrizni a blokklánc érvényességét!
        for i in range(len(self.chain)):
            block = self.chain[i]

            # A) Minden blokk fejléce a benne lévő tranzakciók lenyomatát tartalmazza-e.
            if self.create_hashhexvalue(block['transactions']) != block['block_header']['transactions_hash']:
                raise Exception("Block transactions not valid")

            # B) Az adott blokk fejlécének lenyomata valóban "0000"-val kezdődik-e.
            if self.create_hashhexvalue(block["block_header"])[:4] != "0000":
                raise Exception("Block header not valid")

            # C) Egy blokk fejlécében szereplő hivatkozás az előző blokkra tényleg a hivatkozott blokk fejlécének lenyomata-e.
            if i == 0:
                if block["block_header"]["previous_block_header_hash"] != "ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff":
                    raise Exception("Blockchain not valid")

            else:
                prev_block = self.chain[i-1]
                if block["block_header"]["previous_block_header_hash"] != self.create_hashhexvalue(prev_block["block_header"]):
                    raise Exception("Blockchain not valid")

        return

    def load_pending_transactions(self):
       fileobject = open(self.pending_transactions_filename, 'rb')
       self.pending_transactions = pickle.load(fileobject)
       validated_pending_transactions = []
        # Itt meg kell keresni az unspent outputokat!
        
       while len(self.pending_transactions) != 0:
            transaction = self.pending_transactions.pop() # Végigmegyünk minden tranzakción
            valid = True

            # A) Minden tranzakcióban az input és output összegek megegyeznek-e.
            sum = 0
            for input in transaction['inputs']:
                sum += input[2]
            for output in transaction['outputs']:
                sum -= output['csaposhi']
            if sum != 0:
                t_is_valid = False

            # B) Az input még nem volt elköltve (ehhez előzőleg le kell válogatni az unspent outputokat).
            for input in transaction['inputs']:
                spent_output = [input[0], input[1], input[2]]
                if spent_output not in self.unspent_outputs:
                    t_is_valid = False

            # C) Az inputban szereplő aláírás a publikus kulccsal érvényes aláírásellenőrzést eredményez-e.  
            for input in transaction['inputs']:
                if not self.verify_signature(input[0], input[3], RSA.import_key(input[4])): # input[3] az aláírás érték base64 kódolással
                  t_is_valid = False                                                           # input[4] a publikus kulcsom PEM formátumban

            # D) Az inputban szereplő publikus kulcs lenyomata megegyezik-e a felhasznált outputban megjelölt címzett azonosítóval.
            for input in transaction['inputs']:                                 # A jelenlegi tranzakció minden egyes inputján végigmegyünk
                input_txid = input[0]
                input_idx = input[1]
                for b in self.chain:                                            # Az egész block chain-en végigmegyünk
                    for t in b['transactions']:                                 # És megvizsgálunk minden tranzakciót a blokkon belül
                        if t["txid"] == input_txid:
                            recip = t['outputs'][input_idx]['recipient']        # A tranzakcióból "elrakjuk" a recipient adatait
                            if recip != self.create_hashhexvalue(input[4]):     # Megnézzük, hogy a az inputban szereplő publikus kulcs lenyomata megegyezik-e 
                                valid = False                                   # a felhasznált outputban megjelölt címzett azonosítóval

                pprint("important input:")
                pprint(input)

            if valid: 
                validated_pending_transactions.append(transaction) # Ha sikeres az ellenőrzés, akkor a validáltak közé felveszi.  
            else:
                pprint("invalid tansaction")                       # A nem érvényeseket eldobja, de nem áll le.

       self.pending_transactions = validated_pending_transactions  # Ha kész vagyunk, visszaírjuk a validáltakat az eredeti lista helyére.
        #pprint(self.pending_transactions)
       return

    def find_unspent_outputs(self, payer):
        payer_id = self.load_id (payer)
        self.unspent_outputs = []
        self.payer_unspent_outputs = []
        for block in self.chain:
            for transaction in block['transactions']:
                for output in transaction['outputs']:
                    self.unspent_outputs.append([transaction['txid'], transaction['outputs'].index(output), output['csaposhi']])
                    if output['recipient'] == payer_id:
                        self.payer_unspent_outputs.append([transaction['txid'], transaction['outputs'].index(output), output['csaposhi']])
                for input in transaction['inputs']:
                    if input[0] != 'reward' :     
                      spent_output = [input[0], input[1], input[2]] # A remove() hibát dob, ha úgy törlünk a listából valamit, hogy nem is volt benne,
                      # ezért az input-ból ki kell hagyni azokat az elemeket, amiket az unspent_outputba nem tettünk bele (aláírás érték és publikus kulcs) 
                      pprint(spent_output)
                      self.unspent_outputs.remove(spent_output) # minden input biztosan szerepelt a lánc korábbi outputjaként
                      if input in self.payer_unspent_outputs:
                          self.payer_unspent_outputs.remove(spent_output)
        pprint(self.unspent_outputs)
        pprint(self.payer_unspent_outputs)
        return

    def mine(self, miner):
        if len(self.chain) == 0:
            previous_block_header_hash = 'ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
        else:
            previous_block = self.chain[-1]
            previous_block_header = previous_block['block_header']
            previous_block_header_hash = self.create_hashhexvalue(previous_block_header)

        nonce = 0
        self.load_pending_transactions ()
        reward = self.reward_transaction(len(self.chain), 500, miner) # A lánc hossza bekerül a reward input részébe, hogy biztos egyedi legyen a tranzakció
        # pprint(self.pending_transactions)
        
        block_header = {
            'nonce': nonce,
            'previous_block_header_hash': previous_block_header_hash,
            'transactions_hash' : self.create_hashhexvalue(self.pending_transactions),
        }

        while True:
            block_header_hash = self.create_hashhexvalue(block_header)
            if block_header_hash[:4] == "0000":
                break
            block_header['nonce'] += 1
        # pprint (block_header_hash)

        block = {
            'block_header': block_header,
            'transactions': self.pending_transactions
        }
        
        pprint(block)
        self.chain.append(block)
        # pprint (self.chain)
        self.save_list(self.chain, self.chain_filename)
        self.pending_transactions = [] # Ki kell üríteni a tranzakciós listát, mert bekerültek a blokkba
        self.save_list(self.pending_transactions, self.pending_transactions_filename)
        return

    def generate_first_block(self, miner):
        self.chain = [] # Üres lánccal indulunk
        self.pending_transactions = [] # És üres tranzakció listával
        while len(self.ITKoin_users) > 0:
            recipient_id = self.load_id (self.ITKoin_users.pop()) # előveszi a következő id file nevét és beolvassa az id-t
            for tr in self.pending_transactions: # nem szerepelhet kétszer ugyanaz a recipient, mert akkor a txid azonos lesz
                for op in tr['outputs']:
                    if recipient_id == op['recipient']:
                        pprint ('HIBA: Ismétlődő recipient adatok az első blokk generálásakor.')
                        return False
            self.ICO_transaction(self.ICO, recipient_id)
            #pprint(self.pending_transactions)
        self.save_list(self.pending_transactions, self.pending_transactions_filename)
        self.mine(miner)
        return

    def payment_transaction(self, csaposhi, payer, recipient): # A megadott csaposhi összeg átadása payertől recipientnek és a tranzakció lista bővítése
        payer_id = self.load_id (payer)
        recipient_id = self.load_id (recipient)
        self.find_unspent_outputs(payer)
        sum = 0
        used_outputs=[]
        while (sum < csaposhi):
            next_output=self.payer_unspent_outputs.pop()
            used_outputs.append(next_output)
            sum += next_output[2] # ebben a listapozícióban van a hivatkozott outputban kapott összeg
        inputs = used_outputs
        # Az inputok felhasználási jogának igazolása a felhasználó aláírásával:
        payer_privatekey = self.load_privatekey(payer)
        payer_publickey = self.load_publickeyfromprivatekey(payer)
        for input in inputs: # az inputsban szándékosan nincs benne az akkori recipient, mert ezt abból a tranzakcióból kell majd kivenni és ellenőrizni
            input.append(self.create_signature(input[0], payer_privatekey)) # input[3] az aláírás érték base64 kódolással
            input.append(payer_publickey.export_key().decode('ascii')) # input[4] a publikus kulcsom PEM formátumban
            pprint(RSA.import_key(input[4]))
            pprint(self.verify_signature(input[0], input[3], RSA.import_key(input[4])))
        outputs = [{
            'csaposhi': csaposhi,
            'recipient': recipient_id}]
        if sum > csaposhi: # ha van visszajáró, azt visszautaljuk magunknak
            outputs.append({
                'csaposhi': sum-csaposhi,
                'recipient': payer_id})
        transaction = {
            'inputs': inputs,
            'outputs': outputs}
        transaction ['txid'] = self.create_hashhexvalue(transaction) # a tranzakció lenyomata lesz az azonosítója egyben
        self.pending_transactions.append(transaction)
        self.save_list(self.pending_transactions, self.pending_transactions_filename)
        pprint(self.pending_transactions)
        return

    def reward_transaction(self, length, reward, miner): # A bányászjutalom létrehozása a bányásznak: input tranzakció nincs, de a lánc hossza bekerül az inputba
        miner_id = self.load_id (miner)
        inputs = []
        input = ['reward', length]
        inputs.append(input)
        outputs = [{
            'csaposhi': reward,
            'recipient': miner_id}]
        transaction = {
            'inputs': inputs,
            'outputs': outputs}
        transaction ['txid'] = self.create_hashhexvalue(transaction) # a tranzakció lenyomata lesz az azonosítója egyben
        self.pending_transactions.append(transaction)
        return transaction

    def ICO_transaction(self, ICO, recipient): # Initial Coin Offering kifizetése a résztvevőknek: Input nélküli output
        inputs = []
        outputs = [{
            'csaposhi': ICO,
            'recipient': recipient}]
        transaction = {
            'inputs': inputs,
            'outputs': outputs}
        transaction ['txid'] = self.create_hashhexvalue(transaction) # a tranzakció lenyomata lesz az azonosítója egyben
        self.pending_transactions.append(transaction)
        #pprint(self.pending_transactions)
        return



coin=ITKoin()
coin.create_users()
coin.generate_first_block('Antal')
coin.payment_transaction(200, 'Antal', 'Béla')
coin.mine('Antal')
coin.find_unspent_outputs('Cili')


{'block_header': {'nonce': 99133,
                  'previous_block_header_hash': 'ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
                  'transactions_hash': '4191697a8c90ba5d7023989bcecb320e412ea300eda58b239be61f8d187535da'},
 'transactions': [{'inputs': [],
                   'outputs': [{'csaposhi': 100,
                                'recipient': 'b8794cee2ed56e9fa56705e495e72062658f48f7ef30a8976e708c96a506d355'}],
                   'txid': '36bb0b8a24e581c59edc82811e90c27ada490d22794c020f7ecf13d2091b8d26'},
                  {'inputs': [],
                   'outputs': [{'csaposhi': 100,
                                'recipient': '02b9fac2119cb038ee03fd1d810a361867ea3259d858d795c8a440023be2cc18'}],
                   'txid': '0094d25e88c916893d68551b8947b0cbc445e6edf49d26989a6ef798fe5a28fc'},
                  {'inputs': [],
                   'outputs': [{'csaposhi': 100,
                                'recipient': '3a7361bbc021b471344245bad2bf